## SQL Queries

### Connector

In [3]:
import sqlite3
import pandas as pd

conn=sqlite3.connect("cricsheet_match_data.db")

def run_query(query):
    df=pd.read_sql_query(query,conn)
    return df

### 1) Top 10 batsmen by total runs — ODI

In [27]:
run_query("""
SELECT batter, SUM(runs_batter) AS total_runs
FROM odi_table
GROUP BY batter
ORDER BY total_runs DESC
LIMIT 10;
""")

,batter,total_runs
0,V Kohli,14059
1,KC Sangakkara,11618
2,RG Sharma,10948
3,MS Dhoni,10274
4,AB de Villiers,9435
5,TM Dilshan,9212
6,LRPL Taylor,8126
7,DPMD Jayawardene,8040
8,HM Amla,7834
9,Tamim Iqbal,7648


### 2) Leading wicket-takers — T20

In [28]:
run_query("""
SELECT bowler,
       SUM(CASE WHEN wicket IS NOT NULL AND wicket <> 'None' THEN 1 ELSE 0 END) AS total_wickets
FROM t20_table
GROUP BY bowler
ORDER BY total_wickets DESC
LIMIT 10;
""")

,bowler,total_wickets
0,TG Southee,175
1,ML Schutt,161
2,DB Sharma,156
3,Mustafizur Rahman,155
4,IS Sodhi,152
5,S Ecclestone,149
6,AU Rashid,143
7,Ehsan Khan,138
8,Haris Rauf,138
9,Shakib Al Hasan,138


### 3)Top run-scorers in Test cricket overall

In [29]:
run_query("""
SELECT batter, SUM(runs_batter) AS total_runs
FROM test_table
GROUP BY batter
ORDER BY total_runs DESC
LIMIT 20;
""")

,batter,total_runs
0,JE Root,13543
1,AN Cook,12472
2,SPD Smith,10477
3,KS Williamson,9276
4,V Kohli,9230
5,HM Amla,9146
6,DA Warner,8786
7,KC Sangakkara,8489
8,AB de Villiers,8182
9,AD Mathews,8073


### 4) Highest-scoring venues (all formats combined)

In [30]:
run_query("""
WITH all_deliveries AS (
  SELECT venue, runs_total FROM test_table
  UNION ALL SELECT venue, runs_total FROM odi_table
  UNION ALL SELECT venue, runs_total FROM t20_table
  UNION ALL SELECT venue, runs_total FROM ipl_table
)
SELECT venue, SUM(runs_total) AS runs
FROM all_deliveries
GROUP BY venue
ORDER BY runs DESC
LIMIT 20;
""")

,venue,runs
0,Harare Sports Club,89915
1,Dubai International Cricket Stadium,81115
2,Shere Bangla National Stadium,60880
3,Sydney Cricket Ground,56510
4,Lord's,50574
5,Sheikh Zayed Stadium,50282
6,Melbourne Cricket Ground,47970
7,Galle International Stadium,46906
8,Adelaide Oval,41798
9,Kennington Oval,41775


### 5) Best ODI bowling economy (min 300 balls) — approx (balls counted as deliveries)

In [31]:
run_query("""
SELECT bowler,
       SUM(runs_total) * 6.0 / COUNT(*) AS economy,
       COUNT(*) AS balls
FROM odi_table
GROUP BY bowler
HAVING COUNT(*) >= 300
ORDER BY economy ASC
LIMIT 20;
""")

,bowler,economy,balls
0,T van der Gugten,3.107143,504
1,Shakeel Ahmed,3.118605,860
2,Qanita Jalil,3.176471,323
3,S Molineux,3.294118,663
4,BM Scholtz,3.345330,3223
5,MM Letsoalo,3.358621,870
6,RL Johnson,3.367847,367
7,SM Pollock,3.445169,5061
8,IJR Zwilling,3.448753,722
9,S Loubser,3.450237,633


### 6) Powerplay scoring (overs 0–5) — T20

In [32]:
run_query("""
SELECT team, SUM(runs_total) AS powerplay_runs
FROM t20_table
WHERE over BETWEEN 0 AND 5
GROUP BY team
ORDER BY powerplay_runs DESC
LIMIT 20;
""")

,team,powerplay_runs
0,India,18044
1,England,17482
2,Australia,16775
3,New Zealand,15702
4,Pakistan,15686
5,West Indies,14829
6,South Africa,14452
7,Sri Lanka,13596
8,Bangladesh,11482
9,Ireland,10544


### 7) Death overs economy (overs >=16) — T20 (min 120 balls in 16+)

In [33]:
run_query("""
SELECT bowler,
       SUM(runs_total) * 6.0 / COUNT(*) AS death_economy,
       COUNT(*) AS balls
FROM t20_table
WHERE over >= 16
GROUP BY bowler
HAVING COUNT(*) >= 120
ORDER BY death_economy ASC
LIMIT 20;
""")

,bowler,death_economy,balls
0,Rosine Irera,3.511111,135
1,M Hill,4.220930,172
2,W Mwatile,4.373239,284
3,T Shadrack,4.481481,162
4,D Foerster,4.509317,161
5,I Barma,4.726027,146
6,S Laomi,4.771654,127
7,O Kamchomphu,4.805556,216
8,TT Soe,4.809917,121
9,C Aweko,4.889503,362


### 8) Toss decision distribution — all formats (deliveries proxy)

In [34]:
run_query("""
WITH all_deliveries AS (
  SELECT 'Test' AS fmt, toss_decision FROM test_table
  UNION ALL SELECT 'ODI', toss_decision FROM odi_table
  UNION ALL SELECT 'T20', toss_decision FROM t20_table
  UNION ALL SELECT 'IPL', toss_decision FROM ipl_table
)
SELECT fmt, toss_decision, COUNT(*) AS deliveries
FROM all_deliveries
GROUP BY fmt, toss_decision
ORDER BY fmt, deliveries DESC;
""")

,fmt,toss_decision,deliveries
0,IPL,field,181679
1,IPL,bat,96526
2,ODI,field,800116
3,ODI,bat,798691
4,T20,field,553767
5,T20,bat,504278
6,Test,bat,1197375
7,Test,field,504389


### 9) ODI: cities with the most runs

In [35]:
run_query("""
SELECT city, SUM(runs_total) AS runs
FROM odi_table
GROUP BY city
ORDER BY runs DESC
LIMIT 20;
""")

,city,runs
0,None,132680
1,Colombo,52284
2,London,42706
3,Mirpur,36699
4,Harare,33725
5,Bulawayo,27691
6,Lahore,24834
7,Dubai,23432
8,Karachi,21214
9,Dublin,21014


### 10) Test: most balls faced (approx; counts deliveries)

In [36]:
run_query("""
SELECT batter, COUNT(*) AS balls_faced
FROM test_table
GROUP BY batter
ORDER BY balls_faced DESC
LIMIT 20;
""")

,batter,balls_faced
0,AN Cook,26618
1,JE Root,23599
2,SPD Smith,19579
3,HM Amla,18274
4,KS Williamson,17947
5,Azhar Ali,17063
6,AD Mathews,16743
7,V Kohli,16655
8,CA Pujara,16191
9,KC Sangakkara,15666


### 11) Test: most balls bowled

In [37]:
run_query("""
SELECT bowler, COUNT(*) AS balls_bowled
FROM test_table
GROUP BY bowler
ORDER BY balls_bowled DESC
LIMIT 20;
""")

,bowler,balls_bowled
0,JM Anderson,39114
1,NM Lyon,34507
2,SCJ Broad,33896
3,R Ashwin,27140
4,HMRKB Herath,23563
5,TG Southee,23557
6,I Sharma,19461
7,RA Jadeja,19434
8,MA Starc,19231
9,DW Steyn,18172


### 12) IPL: boundary hitters (4s, 6s, boundary runs)

In [38]:
run_query("""
SELECT batter,
       SUM(CASE WHEN runs_batter = 4 THEN 1 ELSE 0 END) AS fours,
       SUM(CASE WHEN runs_batter = 6 THEN 1 ELSE 0 END) AS sixes,
       (SUM(CASE WHEN runs_batter = 4 THEN 1 ELSE 0 END) * 4
        + SUM(CASE WHEN runs_batter = 6 THEN 1 ELSE 0 END) * 6) AS boundary_runs
FROM ipl_table
GROUP BY batter
ORDER BY boundary_runs DESC
LIMIT 20;
""")

,batter,fours,sixes,boundary_runs
0,V Kohli,774,292,4848
1,RG Sharma,640,303,4378
2,DA Warner,663,236,4068
3,S Dhawan,768,153,3990
4,CH Gayle,408,359,3786
5,SK Raina,506,204,3248
6,AB de Villiers,414,253,3174
7,MS Dhoni,375,264,3084
8,KL Rahul,453,208,3060
9,RV Uthappa,481,182,3016


### 13) IPL: six-hitting rate (sixes per 100 balls) — min 200 balls

In [39]:
run_query("""
SELECT batter,
       SUM(CASE WHEN runs_batter = 6 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS sixes_per_100_balls,
       COUNT(*) AS balls
FROM ipl_table
GROUP BY batter
HAVING COUNT(*) >= 200
ORDER BY sixes_per_100_balls DESC
LIMIT 20
""")

,batter,sixes_per_100_balls,balls
0,J Fraser-McGurk,14.778325,203
1,AD Russell,13.714637,1626
2,N Pooran,11.653873,1433
3,TH David,11.450382,524
4,Ashutosh Sharma,10.937500,256
5,D Brevis,10.819672,305
6,LS Livingstone,10.729614,699
7,R Powell,10.727969,261
8,Rashid Khan,10.594315,387
9,CH Gayle,10.210466,3516


### 14) T20: wicket density by venue (per 100 balls; min 10,000 deliveries)

In [40]:
run_query("""
SELECT venue,
       SUM(CASE WHEN wicket IS NOT NULL AND wicket <> 'None' THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS wickets_per_100_balls,
       COUNT(*) AS deliveries
FROM t20_table
GROUP BY venue
HAVING COUNT(*) >= 10000
ORDER BY wickets_per_100_balls DESC
LIMIT 20;
""")

,venue,wickets_per_100_balls,deliveries
0,Bayer Uerdingen Cricket Ground,6.248140,10083
1,"Gahanga International Cricket Stadium, Rwanda",6.244947,19792
2,"Bayuemas Oval, Kuala Lumpur",5.982819,19673
3,Gahanga International Cricket Stadium. Rwanda,5.873899,16003
4,"Botswana Cricket Association Oval 1, Gaborone",5.797101,11178
5,Entebbe Cricket Oval,5.781383,10603
6,Dubai International Cricket Stadium,5.578910,25453
7,Tribhuvan University International Cricket Gro...,5.511051,10361
8,"West End Park International Cricket Stadium, Doha",5.489135,11459
9,Udayana Cricket Ground,5.488512,14667


### 15) ODI: average runs per delivery by toss decision

In [41]:
run_query("""
SELECT toss_decision,
       AVG(runs_total) AS avg_runs_per_ball,
       COUNT(*) AS deliveries
FROM odi_table
GROUP BY toss_decision
ORDER BY avg_runs_per_ball DESC;
""")

,toss_decision,avg_runs_per_ball,deliveries
0,field,0.832272,800116
1,bat,0.821582,798691


### 16) ODI: team dot-ball rate (min 5,000 balls)

In [42]:
run_query("""
SELECT team,
       SUM(CASE WHEN runs_total = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS dot_ball_pct,
       COUNT(*) AS balls
FROM odi_table
GROUP BY team
HAVING COUNT(*) >= 5000
ORDER BY dot_ball_pct DESC
LIMIT 20;
""")

,team,dot_ball_pct,balls
0,Kenya,60.440813,10526
1,Papua New Guinea,59.661137,16939
2,Canada,59.090081,16463
3,United Arab Emirates,58.584029,29817
4,Oman,57.443114,16700
5,Nepal,57.423780,18663
6,Netherlands,57.283941,26833
7,Namibia,57.121491,17384
8,Zimbabwe,56.614350,81172
9,Ireland,55.432431,49002


### 17) IPL: batter vs bowler head-to-head (min 30 balls)

In [43]:
run_query("""
SELECT batter, bowler,
       COUNT(*) AS balls,
       SUM(runs_batter) AS runs_off_bat
FROM ipl_table
GROUP BY batter, bowler
HAVING COUNT(*) >= 30
ORDER BY runs_off_bat DESC
LIMIT 25;
""")

,batter,bowler,balls,runs_off_bat
0,DA Warner,SP Narine,127,195
1,V Kohli,R Ashwin,157,181
2,V Kohli,RA Jadeja,163,179
3,SK Raina,PP Chawla,102,175
4,RG Sharma,UT Yadav,121,170
5,V Kohli,UT Yadav,98,168
6,V Kohli,A Mishra,108,162
7,KL Rahul,DL Chahar,104,159
8,V Kohli,DJ Bravo,110,157
9,DA Warner,YS Chahal,99,156


### 18) Team with the highest win percentage

In [4]:
run_query("""
WITH matches AS (
  SELECT DISTINCT
         match_id,
         winner,
         teams
  FROM test_table
),
teams_expanded AS (
  -- split "Team A, Team B" into two rows
  SELECT match_id,
         TRIM(SUBSTR(teams, 1, INSTR(teams, ',') - 1)) AS team,
         winner
  FROM matches
  WHERE INSTR(teams, ',') > 0
  UNION ALL
  SELECT match_id,
         TRIM(SUBSTR(teams, INSTR(teams, ',') + 1)) AS team,
         winner
  FROM matches
  WHERE INSTR(teams, ',') > 0
),
per_team AS (
  SELECT
    team,
    COUNT(DISTINCT CASE WHEN winner IS NOT NULL AND winner <> 'draw' THEN match_id END) AS decided_matches,
    COUNT(DISTINCT CASE WHEN winner = team THEN match_id END) AS wins
  FROM teams_expanded
  GROUP BY team
)
SELECT
  team,
  wins,
  decided_matches,
  ROUND(wins * 100.0 / NULLIF(decided_matches,0), 2) AS win_percentage
FROM per_team
WHERE decided_matches > 0
ORDER BY win_percentage DESC, wins DESC
LIMIT 10;
""")

,team,wins,decided_matches,win_percentage
0,Australia,129,191,67.54
1,India,107,165,64.85
2,South Africa,97,160,60.63
3,England,120,218,55.05
4,New Zealand,65,130,50.00
5,Sri Lanka,67,137,48.91
6,Pakistan,55,124,44.35
7,West Indies,37,127,29.13
8,Ireland,2,8,25.00
9,Bangladesh,21,103,20.39


### 19) Total number of centuries across all match types

In [6]:

run_query("""
WITH all_deliveries AS (
  SELECT 'test' AS match_type,
         match_id,
         batter, runs_batter
  FROM test_table
  UNION ALL
  SELECT 'odi',  match_id, batter, runs_batter FROM odi_table
  UNION ALL
  SELECT 't20',  match_id, batter, runs_batter FROM t20_table
  UNION ALL
  SELECT 'ipl',  match_id, batter, runs_batter FROM ipl_table
),
innings AS (
  SELECT match_type, match_id, batter, SUM(runs_batter) AS runs
  FROM all_deliveries
  GROUP BY match_type, match_id, batter
),
centuries AS (
  SELECT match_type, match_id, batter
  FROM innings
  WHERE runs >= 100
)
SELECT match_type, COUNT(*) AS total_centuries
FROM centuries
GROUP BY match_type
ORDER BY total_centuries DESC;
""")

,match_type,total_centuries
0,test,2476
1,odi,1563
2,t20,240
3,ipl,110


### 20) Narrowest margin of victory by runs — all formats (approx per "match key")

In [ ]:
run_query("""
WITH team_totals AS (
  SELECT match_id, season, venue, teams, winner, team, SUM(runs_total) AS team_runs
  FROM test_table
  GROUP BY match_id, season, venue, teams, winner, team
),
per_match AS (
  SELECT match_id, season, venue, teams, winner,
         MAX(team_runs) AS max_runs,
         MIN(team_runs) AS min_runs
  FROM team_totals
  GROUP BY match_id, season, venue, teams, winner
),
winner_row AS (
  SELECT t.match_id, t.team AS winner_team, t.team_runs AS winner_runs
  FROM team_totals t
  JOIN per_match p USING (match_id)
  WHERE t.team = p.winner
)
SELECT
  p.season, p.venue, p.teams, p.winner,
  (p.max_runs - p.min_runs) AS run_margin
FROM per_match p
JOIN winner_row w USING (match_id)
WHERE p.winner IS NOT NULL
  AND p.winner <> 'draw'
  AND w.winner_runs = p.max_runs     -- winner had the higher total (win by runs)
  AND (p.max_runs - p.min_runs) > 0  -- strictly positive margin
ORDER BY run_margin ASC
LIMIT 10;
""")

,season,venue,teams,winner,run_margin
0,2016/17,Adelaide Oval,"Australia, South Africa",Australia,1
1,2016/17,Basin Reserve,"New Zealand, Bangladesh",New Zealand,1
2,2017,Headingley,"England, West Indies",West Indies,1
3,2017,Lord's,"West Indies, England",England,1
4,2016/17,Himachal Pradesh Cricket Association Stadium,"India, Australia",India,1
5,2016/17,P Sara Oval,"Sri Lanka, Bangladesh",Bangladesh,1
6,2018,"The Village, Malahide","Ireland, Pakistan",Pakistan,1
7,2018,"Kensington Oval, Bridgetown","West Indies, Sri Lanka",Sri Lanka,1
8,2018/19,Kingsmead,"South Africa, Sri Lanka",Sri Lanka,1
9,2018/19,St George's Park,"South Africa, Sri Lanka",Sri Lanka,1


In [47]:
conn.close()